## Problem Statement
#### To aid in electricity analysis and budgeting for the buildings owned by the District of Columbia,  predicting the energy use of building based on historical usage and weather data is sought.

## SMART Goals
#### Develop a model to predict a given building's electricity usage for each month in 2017 within 10% of the actual amounts based on historical electricity usage and weather data from the years 2013-2016.
#### Then add the 2017 historical data to the model's training data to predict the usage for 2018.

In [1]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import seaborn as sns
import datetime as dt

# display plots in the notebook
%matplotlib inline

# increase default figure and font sizes for easier viewing
plt.rcParams['figure.figsize'] = (18, 10)
plt.rcParams['font.size'] = 14

In [2]:
# start-time defined as 1/1/2013 00:00:00 ETC
# start_time = 1357016400
# # start-time defined as 12/31/2017 23:45:00 ETC
# end_time = 1514781900

# for simplicity I'm using UTC, but the above could be used if I wanted to use ETC, which is what the DC Gov't operates under
# start-time defined as 1/1/2013 00:00:00 UTC
start_time = 1356998400
# start-time defined as 12/31/2017 23:45:00 UTC
end_time = 1514763900

# insert (list of) building ID(s)
bldg_ids = [255]

In [3]:
def get_bldg_data(bldg):
    temp = requests.get('https://api.newcityenergy.com/v1/buildings/'+str(bldg)+'/interval_data?start_time='+str(start_time)+'&end_time='+str(end_time)+'&data_quality=clean')
    tempdf = pd.DataFrame.from_dict(temp.json(),orient='index')
    tempindex = []
    for x in temp.json().keys():
        x = float(x)
        x = int(x)
        tempindex.append(x)
    tempdf.set_index(pd.DataFrame(tempindex)[0], inplace=True)
    tempdf.rename(columns={0:str(bldg)+'_elec_use'}, inplace=True)
    tempdf.index = pd.to_datetime(tempindex, unit='s')
    tempdf.sort_index(inplace=True)
    return tempdf

In [4]:
bldg_data = pd.DataFrame()
for b in bldg_ids:
    bldg_data = bldg_data.merge(get_bldg_data(b),left_index=True,right_index=True,how='outer')
bldg_data.index.rename('UTC_Datetime', inplace=True)
bldg_data.fillna(0, inplace=True)

In [5]:
bldg_data.head()

,255_elec_use
UTC_Datetime,
2013-01-01 00:00:00,314.55
2013-01-01 00:15:00,329.47
2013-01-01 00:30:00,341.44
2013-01-01 00:45:00,327.38
2013-01-01 01:00:00,314.87


In [6]:
bldg_data.groupby([bldg_data.index.date,bldg_data.index.hour]).sum().head(25)
# I may want to get rid of the multi index, but definitely want to aggregate data to hourly, daily or monthly at the very least.

255_elec_use
           UTC_Datetime              
2013-01-01 0                  1312.84
           1                  1327.41
           2                  1293.80
           3                  1306.04
           4                  1273.91
           5                  1294.75
           6                  1305.70
           7                  1305.92
           8                  1282.77
           9                  1270.71
           10                 1293.28
           11                 1298.50
           12                 1282.48
           13                 1255.22
           14                 1295.86
           15                 1267.43
           16                 1281.86
           17                 1272.74
           18                 1266.96
           19                 1258.27
           20                 1273.42
           21                 1251.36
           22                 1245.31
           23                 1278.61
2013-01-02 0                  1274.02

In [7]:
# I downloaded historical weather data for Washington, DC and saved it here: Historical_Weather_Data.csv
weather_data = pd.read_csv('Historical_Weather_Data.csv',index_col=0,parse_dates=True)
weather_data = weather_data.fillna(0)
weather_data.index = pd.to_datetime(weather_data.index, unit='s')

In [8]:
weather_data.head()

,dt_iso,city_id,city_name,lat,lon,temp,temp_min,temp_max,pressure,sea_level,...,rain_today,snow_1h,snow_3h,snow_24h,snow_today,clouds_all,weather_id,weather_main,weather_description,weather_icon
dt,,,,,,,,,,,,,,,,,,,,,
2012-10-01 13:00:00,2012-10-01 13:00:00 +0000 UTC,4366164,0.0,0.0,0.0,285.64,283.15,287.15,1015,0.0,...,0.0,0.0,0.0,0.0,0.0,90,804,Clouds,overcast clouds,04d
2012-10-01 14:00:00,2012-10-01 14:00:00 +0000 UTC,4366164,0.0,0.0,0.0,287.53,284.15,289.82,1015,0.0,...,0.0,0.0,0.0,0.0,0.0,1,800,Clear,sky is clear,01d
2012-10-02 15:00:00,2012-10-02 15:00:00 +0000 UTC,4366164,0.0,0.0,0.0,290.61,288.15,292.15,1016,0.0,...,0.0,0.0,0.0,0.0,0.0,75,803,Clouds,broken clouds,04d
2012-10-02 16:00:00,2012-10-02 16:00:00 +0000 UTC,4366164,0.0,0.0,0.0,292.45,291.15,293.15,1015,0.0,...,0.0,0.0,0.0,0.0,0.0,75,803,Clouds,broken clouds,04d
2012-10-02 17:00:00,2012-10-02 17:00:00 +0000 UTC,4366164,0.0,0.0,0.0,293.42,292.15,295.15,1015,0.0,...,0.0,0.0,0.0,0.0,0.0,77,300,Drizzle,light intensity drizzle,09d


In [9]:
weather_features = pd.DataFrame(index=weather_data.index)

In [10]:
weather_features.head()

""
dt
2012-10-01 13:00:00
2012-10-01 14:00:00
2012-10-02 15:00:00
2012-10-02 16:00:00
2012-10-02 17:00:00


In [11]:
# Calculating Heating Degree Days and Cooling Degree Days to quantify heating and cooling needs of the buildings
weather_features['HDD'] = weather_data.temp.apply(lambda x: np.max([(291.483-x)/24,0]))
weather_features['CDD'] = weather_data.temp.apply(lambda x: np.max([(x-291.483)/24,0]))

In [12]:
weather_features.head()

,HDD,CDD
dt,,
2012-10-01 13:00:00,0.243458,0.000000
2012-10-01 14:00:00,0.164708,0.000000
2012-10-02 15:00:00,0.036375,0.000000
2012-10-02 16:00:00,0.000000,0.040292
2012-10-02 17:00:00,0.000000,0.080708


In [13]:
weather_features = weather_features['20130101':'20171231']

In [14]:
weather_features.head()

,HDD,CDD
dt,,
2013-01-01 00:00:00,0.702208,0.0
2013-01-01 01:00:00,0.730542,0.0
2013-01-01 02:00:00,0.764292,0.0
2013-01-01 03:00:00,0.778875,0.0
2013-01-01 04:00:00,0.799708,0.0


In [15]:
# consider adding a work_day column that is 1 if work day or 0 if not